In [45]:
!pip install -U kaleido
!pip install plotly==5.15.0

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.5/15.5 MB 3.5 MB/s eta 0:00:00:00:0100:01
  Attempting uninstall: plotly
    Found existing installation: plotly 5.18.0
    Uninstalling plotly-5.18.0:
      Successfully uninstalled plotly-5.18.0


In [39]:
import plotly
import pandas as pd
import numpy as np

In [40]:
data = pd.read_csv("../ikresults/results_input_variations_2.csv")
data

,device_name,data_size,joints_scale,architecture,network,layers,neurons,optimizer,loss,completed_epochs,...,pi_percent_5(deg),min_ya(deg),avg_ya(deg),max_ya(deg),std_ya(deg),ya_percent_1(deg),ya_percent_2(deg),ya_percent_3(deg),ya_percent_4(deg),ya_percent_5(deg)
0,NVIDIA GeForce RTX 3090,1000000,10,"MLP [6, 1024, 1024, 1024, 1024, 1024, 1024, 10...",MLP,12,1024,Adam,lq,999,...,9.750000,0.002305,81.076294,179.994583,52.059708,0.760000,1.522222,2.271111,3.021111,3.747778
1,NVIDIA GeForce RTX 2080 Ti,1000000,10,"MLP [19, 1024, 7]",MLP,1,1024,Adam,lq,999,...,9.122222,0.001753,75.325226,179.999435,49.546028,0.713333,1.476667,2.260000,3.021111,3.746667
2,NVIDIA GeForce RTX 3090,1000000,10,"ResMLP [6, 1024, 5, 7]",ResMLP,1,1024,Adam,lq,999,...,11.372222,0.000350,69.813354,179.999786,52.100471,1.077778,2.185556,3.336667,4.461111,5.558889
3,NVIDIA GeForce RTX 2080 Ti,1000000,10,"ResMLP [19, 1024, 5, 7]",ResMLP,1,1024,Adam,lq,999,...,99.997778,0.000012,0.301581,8.321875,0.305597,97.228889,99.658889,99.908889,99.974444,99.990000
4,NVIDIA GeForce RTX 3090,1000000,10,"DenseMLP [6, 1024, 5, 7]",DenseMLP,1,1024,Adam,lq,999,...,10.415556,0.000023,73.777336,179.998810,52.159649,0.998889,1.915556,2.917778,3.917778,4.872222
5,NVIDIA GeForce RTX 3090,1000000,10,"DenseMLP [19, 1024, 5, 7]",DenseMLP,1,1024,Adam,lq,999,...,99.996667,0.000003,0.194130,6.577154,0.200518,99.187778,99.916667,99.976667,99.994444,99.997778


In [41]:
data_plot_pos = []
data_plot_pos_min = []
data_plot_pos_max = []
data_plot_ori = []

layers_pose = pd.DataFrame(data, columns = ['average_position_error(mm)', 'average_orientation_error(deg)'])
data_plot_pose = layers_pose.to_numpy()

layers_pos = pd.DataFrame(data, columns = ['avg_x(mm)', 'avg_y(mm)', 'avg_z(mm)'])
data_plot_pos = layers_pos.to_numpy()

layers_ori = pd.DataFrame(data, columns = ['avg_ro(deg)', 'avg_pi(deg)', 'avg_ya(deg)'])
data_plot_ori = layers_ori.to_numpy()

In [44]:
from plotly.subplots import make_subplots
import plotly.graph_objects as go

networks = ["MLP(R)", "MLP(T)", "ResMLP(R)", "ResMLP(T)", "DenseMLP(R)", "DenseMLP(T)"]
plot_names_1 = ["$x$", "$y$", "$z$"]
plot_names_2 = ["$ro$", "$pi$", "$ya$"]
#data_plot = np.array(data_plot)
#colors = ["orange", "lightsalmon", "indianred", "lightsalmon", "indianred", "blue", "indianred", "cadetblue", "indianred", "darkgray"]
colors = [
    '#1f77b4',  # muted blue
    '#ff7f0e',  # safety orange
    '#2ca02c',  # cooked asparagus green
    '#d62728',  # brick red
    '#9467bd',  # muted purple
    '#8c564b',  # chestnut brown
    '#e377c2',  # raspberry yogurt pink
    '#7f7f7f',  # middle gray
    '#bcbd22',  # curry yellow-green
    '#17becf'   # blue-teal
]
neurons = 1024

#fig = go.Figure()

fig = make_subplots(rows=1, cols=2) #, subplot_titles=("Position", "Orientation"))

### plot the position errors
for i in range(6):
  fig.add_trace(go.Bar(
      x=plot_names_1,
      y=data_plot_pos[i,:],
      name=networks[i],
      marker_color=colors[i],
      #text=data_plot[:,i],
      #textposition='auto',
  ), row=1, col=1)


  fig.add_trace(go.Bar(
      x=plot_names_2,
      y=data_plot_ori[i,:],
      #name=networks[i],
      marker_color=colors[i],
      #text=data_plot[:,i],
      #textposition='auto',
      showlegend=False
  ), row=1, col=2)



# Here we modify the tickangle of the xaxis, resulting in rotated labels.
fig.update_layout(barmode='group',
                  plot_bgcolor='white',
                  xaxis_tickangle=0,
                  font=dict(size=15),
                  #legend_title_text='Neurons:',
                  title='Position and Orientation Mean Absolute Errors',
                  #xaxis_title='Position',
                  #yaxis_title='Position errors (mm)',
                  yaxis_range=[0,200],
                  width=1200,
                  height=500,
                  legend=dict(
                      yanchor="top",
                      y=1.13,
                      xanchor="left",
                      x=0.0,
                      orientation="h",
                      font_size = 13
                  )
)

fig.update_xaxes(
    mirror=True,
    ticks='outside',
    showline=True,
    linecolor='black',
    gridcolor='lightgrey',
)
fig.update_yaxes(
    mirror=True,
    ticks='outside',
    showline=True,
    linecolor='black',
    gridcolor='lightgrey'
)

fig.update_yaxes(title_text="Position errors (mm)", row=1, col=1)
fig.update_yaxes(title_text="Orientation errors (deg)", row=1, col=2)

fig.update_traces(texttemplate='%{y:.1f}', textposition='outside', textfont_size=10, textangle=0)
fig.show()
#fig.write_image("../ikresults/all_errors_1.pdf", engine="kaleido")
#fig.to_image(format="pdf", engine="kaleido")
